<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/6_versiyon_deneme_2_gelistir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
# DOSYA ADI: teknofest_zorlu_egitim.py
import numpy as np
import pandas as pd
import pickle
import joblib
import xgboost as xgb
import lightgbm as lgb
import os
import tensorflow as tf
import random

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

# --- AYARLAR ---
MAX_EPOCH_LIMIT = 50
BATCH_SIZE = 32

# Rastgelelikleri sabitle
np.random.seed(42)
tf.random.set_seed(42)

print(">> SİSTEM BAŞLATILIYOR...")

# --- KULLANICIYA SORU ---
print("\n" + "="*50)
print("🤔 EĞİTİM STRATEJİSİ BELİRLEME")
muallak_sayisi = input(">> Kaç adet 'MUALLAK' (Arada kalmış/Zor) veri üreteyim? (Örn: 500): ")
try:
    muallak_sayisi = int(muallak_sayisi)
except:
    muallak_sayisi = 0
    print(">> Geçersiz sayı girildi, 0 kabul edildi.")
print("="*50 + "\n")

# --- 1. FONKSİYONLAR ---

# A) Standart Temiz Veri (Kolay/Net Örnekler)
def veri_uret_standart(n_patojenik, n_benign, kategori_adi):
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
    total = n_patojenik + n_benign

    for i in range(total):
        label = 1 if i < n_patojenik else 0
        dna = ''.join(np.random.choice(list('ACGT'), size=11))
        prot = ''.join(np.random.choice(amino_acids, size=11))

        # Net ayrımlar (Hasta ise Risk yüksek, Değilse düşük)
        if label == 1:
            risk = np.random.beta(5, 2)
            cons = np.random.uniform(6, 10)
        else:
            risk = np.random.beta(2, 5)
            cons = np.random.uniform(0, 4)

        maf = np.random.exponential(0.05)
        hydro = np.random.uniform(-5, 5)
        polar = np.random.uniform(-3, 3)
        weight = np.random.uniform(-50, 50)

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)
    return dna_seqs, prot_seqs, bio_feats, labels

# B) MUALLAK VERİ (Modelin kafasını karıştıracak zor örnekler)
def veri_uret_muallak(adet):
    print(f"   ⚠️ {adet} adet 'Muallak/Zor' veri üretiliyor...")
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    for _ in range(adet):
        # Ortada kalan özellikler (Gri Bölge)
        risk = np.random.uniform(0.4, 0.7) # Tam ortada riskler
        cons = np.random.uniform(4, 7)     # Tam ortada conservation
        maf = np.random.uniform(0.01, 0.1)
        hydro = np.random.uniform(-2, 2)
        polar = np.random.uniform(-1, 1)
        weight = np.random.uniform(-10, 10)

        # Etiketi formülle belirle ama GÜRÜLTÜ ekle (Bazen yanlış etiketlensin gibi)
        score = (risk * 0.5) + (cons/10 * 0.3) + np.random.normal(0, 0.15)
        label = 1 if score > 0.6 else 0

        dna = ''.join(np.random.choice(list('ACGT'), size=11))
        prot = ''.join(np.random.choice(amino_acids, size=11))

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)

    return dna_seqs, prot_seqs, bio_feats, labels

# --- 2. VERİ SETLERİNİ BİRLEŞTİR ---
def veri_hazirla():
    all_dna, all_prot, all_feats, all_labels = [], [], [], []

    # 1. Standart Veriler (Senin verdiğin sayılar)
    train_dagilim = {"Genel": (1500, 1500), "Kanser": (200, 200), "PAH": (200, 200), "CFTR": (70, 70)}
    print(">> Standart Veriler Hazırlanıyor...")
    for _, (p, b) in train_dagilim.items():
        d, pr, f, l = veri_uret_standart(p, b, "")
        all_dna.extend(d); all_prot.extend(pr); all_feats.extend(f); all_labels.extend(l)

    # 2. Muallak Verileri Ekle
    if muallak_sayisi > 0:
        d_m, p_m, f_m, l_m = veri_uret_muallak(muallak_sayisi)
        all_dna.extend(d_m); all_prot.extend(p_m); all_feats.extend(f_m); all_labels.extend(l_m)

    # Karıştır
    combined = list(zip(all_dna, all_prot, all_feats, all_labels))
    np.random.shuffle(combined)
    dna, prot, feats, labels = zip(*combined)

    return list(dna), list(prot), np.array(feats).astype('float32'), np.array(labels)

# Veriyi Çek
dna_train, prot_train, num_train, y_train = veri_hazirla()
print(f">> TOPLAM EĞİTİM VERİSİ: {len(y_train)} adet.")

# --- 3. TOKENIZATION ---
dna_tok = Tokenizer(char_level=True)
dna_tok.fit_on_texts(dna_train)
prot_tok = Tokenizer(char_level=True)
prot_tok.fit_on_texts(prot_train)

X_dna = pad_sequences(dna_tok.texts_to_sequences(dna_train), maxlen=11, padding='post')
X_prot = pad_sequences(prot_tok.texts_to_sequences(prot_train), maxlen=11, padding='post')

# --- 4. MODEL EĞİTİMİ (Kısaltılmış) ---
X_dna_base, X_dna_meta, X_prot_base, X_prot_meta, X_num_base, X_num_meta, y_base, y_meta = train_test_split(
    X_dna, X_prot, num_train, y_train, test_size=0.2, random_state=42
)

if not os.path.exists('model_stack'): os.makedirs('model_stack')

# CNN
print("\n>> [CNN] Eğitiliyor (Zor verilere karşı direnç kazanıyor)...")
in_dna = Input(shape=(11,)); emb_dna = Embedding(len(dna_tok.word_index)+1, 8)(in_dna)
x1 = GlobalMaxPooling1D()(Conv1D(32, 3, activation='relu')(emb_dna))
in_prot = Input(shape=(11,)); emb_prot = Embedding(len(prot_tok.word_index)+1, 8)(in_prot)
x2 = GlobalMaxPooling1D()(Conv1D(32, 3, activation='relu')(emb_prot))
in_num = Input(shape=(6,)); x3 = BatchNormalization()(Dense(32, activation='relu')(in_num))
merged = Concatenate()([x1, x2, x3])
out = Dense(1, activation='sigmoid')(Dropout(0.5)(Dense(64, activation='relu')(merged)))

model_cnn = Model(inputs=[in_dna, in_prot, in_num], outputs=out)
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Best Epoch Callback
class BestEpoch(Callback):
    def __init__(self):
        self.best_f1 = 0
    def on_epoch_end(self, epoch, logs=None):
        val_p = (self.model.predict([X_dna_meta, X_prot_meta, X_num_meta], verbose=0) > 0.5).astype(int)
        f1 = f1_score(y_meta, val_p)
        if f1 > self.best_f1:
            self.best_f1 = f1
            self.model.save('model_stack/best_cnn_hard.h5')
            print(f"   * Ep {epoch+1}: F1 %{f1*100:.2f} (Kaydedildi)")

model_cnn.fit([X_dna_base, X_prot_base, X_num_base], y_base, epochs=MAX_EPOCH_LIMIT, batch_size=32, verbose=0, callbacks=[BestEpoch()])
model_cnn = load_model('model_stack/best_cnn_hard.h5')

# XGB / LGBM
print(">> [XGB & LGBM] Eğitiliyor...")
model_xgb = xgb.XGBClassifier(n_estimators=100, max_depth=5).fit(X_num_base, y_base)
model_lgbm = lgb.LGBMClassifier(n_estimators=100, verbose=-1).fit(X_num_base, y_base)

# META
print(">> [META] Modeller Birleştiriliyor...")
p_cnn = model_cnn.predict([X_dna_meta, X_prot_meta, X_num_meta], verbose=0).flatten()
p_xgb = model_xgb.predict_proba(X_num_meta)[:, 1]
p_lgbm = model_lgbm.predict_proba(X_num_meta)[:, 1]
meta_model = LogisticRegression().fit(np.column_stack((p_cnn, p_xgb, p_lgbm)), y_meta)

# KAYDET
joblib.dump(meta_model, 'model_stack/meta_model_final.pkl')
joblib.dump(model_xgb, 'model_stack/xgb_final.pkl')
joblib.dump(model_lgbm, 'model_stack/lgbm_final.pkl')
with open('model_stack/dna_tok_final.pickle', 'wb') as f: pickle.dump(dna_tok, f)
with open('model_stack/prot_tok_final.pickle', 'wb') as f: pickle.dump(prot_tok, f)

print(f"\n✅ ZORLU EĞİTİM TAMAMLANDI. (Eklenen Muallak Veri: {muallak_sayisi})")

>> SİSTEM BAŞLATILIYOR...

🤔 EĞİTİM STRATEJİSİ BELİRLEME
>> Kaç adet 'MUALLAK' (Arada kalmış/Zor) veri üreteyim? (Örn: 500): 30000

>> Standart Veriler Hazırlanıyor...
   ⚠️ 30000 adet 'Muallak/Zor' veri üretiliyor...
>> TOPLAM EĞİTİM VERİSİ: 33940 adet.

>> [CNN] Eğitiliyor (Zor verilere karşı direnç kazanıyor)...


   * Ep 1: F1 %40.39 (Kaydedildi)


   * Ep 2: F1 %43.51 (Kaydedildi)


   * Ep 3: F1 %43.80 (Kaydedildi)


   * Ep 4: F1 %45.19 (Kaydedildi)


...

   * Ep 26: F1 %45.91 (Kaydedildi)


   * Ep 41: F1 %46.01 (Kaydedildi)


   * Ep 45: F1 %46.02 (Kaydedildi)


>> [XGB & LGBM] Eğitiliyor...
>> [META] Modeller Birleştiriliyor...

✅ ZORLU EĞİTİM TAMAMLANDI. (Eklenen Muallak Veri: 30000)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [108]:
# DOSYA ADI: canli_test_zorlu.py
import numpy as np
import pickle
import joblib
import time
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Renkler
YESIL = "\033[92m"
KIRMIZI = "\033[91m"
MAVI = "\033[94m"
RESET = "\033[0m"

print(">> CANLI TEST BAŞLIYOR (Random & Formula Based)...")

# --- 1. MODELLERİ YÜKLE ---
try:
    with open('model_stack/dna_tok_final.pickle', 'rb') as f: dna_tok = pickle.load(f)
    with open('model_stack/prot_tok_final.pickle', 'rb') as f: prot_tok = pickle.load(f)
    model_cnn = load_model('model_stack/best_cnn_hard.h5') # Yeni isim
    model_xgb = joblib.load('model_stack/xgb_final.pkl')
    model_lgbm = joblib.load('model_stack/lgbm_final.pkl')
    meta_model = joblib.load('model_stack/meta_model_final.pkl')
except:
    print("❌ Lütfen önce 'teknofest_zorlu_egitim.py' dosyasını çalıştır!")
    exit()

# --- 2. TAMAMEN RANDOM VERİ ÜRETİCİ ---
def tamamen_random_veri_uret():
    # 1. Hiçbir kural gözetmeksizin sayıları salla
    risk = np.random.uniform(0, 1)   # 0 ile 1 arası herhangi bir şey
    cons = np.random.uniform(0, 10)  # 0 ile 10 arası herhangi bir şey
    maf = np.random.exponential(0.05)
    hydro = np.random.uniform(-5, 5)
    polar = np.random.uniform(-3, 3)
    weight = np.random.uniform(-50, 50)

    # 2. DNA/Protein salla
    dna = ''.join(np.random.choice(list('ACGT'), size=11))
    prot = ''.join(np.random.choice(list("ACDEFGHIKLMNPQRSTVWY"), size=11))

    # 3. DOĞA KANUNU (Ground Truth Hesaplama)
    # Modelin bilmediği gizli formül bu. Bakalım model bunu tahmin edebilecek mi?
    # Skor hesapla
    score = (risk * 0.45) + (cons/10 * 0.25) + ((0.5 - maf) * 0.1) + (abs(hydro)/5 * 0.1)

    # Eğer skor 0.60 ile 0.70 arasındaysa "Muallak" durumdur.
    muallak_mi = 0.60 < score < 0.70

    gercek_etiket = 1 if score > 0.65 else 0

    return dna, prot, [risk, maf, cons, hydro, polar, weight], gercek_etiket, muallak_mi

# --- 3. TAHMİN MEKANİZMASI ---
def tahmin_et(dna, prot, feats):
    s_d = pad_sequences(dna_tok.texts_to_sequences([dna]), maxlen=11, padding='post')
    s_p = pad_sequences(prot_tok.texts_to_sequences([prot]), maxlen=11, padding='post')
    f_n = np.array([feats]).astype('float32')

    p_cnn = model_cnn.predict([s_d, s_p, f_n], verbose=0).flatten()[0]
    p_xgb = model_xgb.predict_proba(f_n)[:, 1][0]
    p_lgbm = model_lgbm.predict_proba(f_n)[:, 1][0]

    inp = np.array([[p_cnn, p_xgb, p_lgbm]])
    return meta_model.predict(inp)[0], meta_model.predict_proba(inp)[:, 1][0]

# --- 4. SİMÜLASYON ---
dogru, yanlis = 0, 0
print(f"\n{'DURUM':<10} | {'RİSK':<5} {'CONS':<5} | {'GERÇEK':<8} | {'TAHMİN':<8} | {'SONUÇ'}")
print("-" * 65)

for i in range(25): # 25 Test yapalım
    dna, prot, feats, gercek, muallak = tamamen_random_veri_uret()
    pred, prob = tahmin_et(dna, prot, feats)

    # Görsellik
    str_gercek = "HASTA" if gercek == 1 else "SAĞLAM"
    str_tahmin = "HASTA" if pred == 1 else "SAĞLAM"

    if gercek == pred:
        ikon = f"{YESIL}✅{RESET}"
        dogru += 1
    else:
        ikon = f"{KIRMIZI}❌{RESET}"
        yanlis += 1

    # Eğer muallak (zor) veriyse yanına işaret koyalım
    zorluk = f"{MAVI}[ZOR]{RESET}" if muallak else "     "

    print(f"{zorluk:<10} | {feats[0]:.2f}  {feats[2]:.1f}   | {str_gercek:<8} | {str_tahmin:<8} | {ikon}")
    time.sleep(0.1)

print("-" * 65)
print(f"📊 SONUÇ: {dogru} Doğru / {yanlis} Yanlış")
print(f"📈 BAŞARI: %{(dogru/(dogru+yanlis))*100:.2f}")

>> CANLI TEST BAŞLIYOR (Random & Formula Based)...

DURUM      | RİSK  CONS  | GERÇEK   | TAHMİN   | SONUÇ
-----------------------------------------------------------------


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


           | 0.22  6.8   | SAĞLAM   | HASTA    | ❌


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


           | 0.87  0.6   | SAĞLAM   | SAĞLAM   | ✅


...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[ZOR] | 0.98  5.3   | SAĞLAM   | HASTA    | ❌


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


           | 0.58  8.0   | SAĞLAM   | HASTA    | ❌
[ZOR] | 0.77  9.2   | HASTA    | HASTA    | ✅
-----------------------------------------------------------------
📊 SONUÇ: 13 Doğru / 12 Yanlış
📈 BAŞARI: %52.00


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
